In [100]:
import numpy as np
import re
from operator import itemgetter

In [101]:
Corpus = {'l o w _' : 5,
          'l o w e r _' : 2,
          'n e w e s t _' : 6,
          'w i d e s t _' : 3,
          'h a p p i e r _' : 2}

Corpus

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w e s t _': 6,
 'w i d e s t _': 3,
 'h a p p i e r _': 2}

In [102]:
def getPairCounts(Corpus):
    pairs = {}
    for word, fr in Corpus.items():
        symbols = word.split(' ')
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i+1])
            cfr = pairs.get(pair, 0)
            pairs[pair] = cfr + fr
    return pairs

In [103]:
pairsCounts = getPairCounts(Corpus)

In [104]:
pairsCounts

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}

In [105]:
def getBestPair(pairsCounts):
    return max(pairsCounts, key=pairsCounts.get)

In [106]:
bestPair = getBestPair(pairsCounts)

In [107]:
def mergeInCorpus(bestPair, Corpus):
    newCorpus = {}
    for word in Corpus:
        newWord = re.sub(' '.join(bestPair), ''.join(bestPair), word)
        newCorpus[newWord] = Corpus[word]
    return newCorpus

In [108]:
mergeInCorpus(bestPair, Corpus)

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w es t _': 6,
 'w i d es t _': 3,
 'h a p p i e r _': 2}

In [109]:
def runBPE(Corpus, k):
    
    bpeStats = {}
    for i in range(k):
        pairCounts = getPairCounts(Corpus)
        if not pairCounts:
            breka
        bestPair = getBestPair(pairCounts)
        bpeStats[bestPair] = i
        Corpus = mergeInCorpus(bestPair, Corpus)
    return Corpus, bpeStats

In [110]:
newCorpus, bpeStats = runBPE(Corpus, 10)

In [111]:
newCorpus

{'low_': 5, 'low er _': 2, 'newest_': 6, 'w i d est_': 3, 'h a p p i er _': 2}

In [112]:
bpeStats

{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '_'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est_'): 7,
 ('low', '_'): 8,
 ('e', 'r'): 9}

In [113]:
def getAllPairs(word):
    pairs = []
    word = word.split(' ')
    prevChar = word[0]
    for char in word[1:]:
        pairs.append((prevChar, char))
        prevChar = char
    return pairs

In [114]:
newWord2 = 'lowest'
newWord2 = ' '.join(list(newWord2)) + ' _'

In [115]:
pairs = getAllPairs(newWord2)

In [116]:
pairs

[('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 's'), ('s', 't'), ('t', '_')]

In [117]:
bpeCodes = [(pair, bpeStats[pair]) for pair in pairs if pair in bpeStats]

In [118]:
def getPairToBeMerged(bpeStats, pairs):
    bpeCodes = [(pair, bpeStats[pair]) for pair in pairs if pair in bpeStats]
    if len(bpeCodes) == 0:
        return -1
    pairToBeMerged = min(bpeCodes, key=itemgetter(1))[0]
    return pairToBeMerged

In [119]:
pairToBeMerged = getPairToBeMerged(bpeStats, pairs)

In [120]:
def mergeLetters(word, pairToBeMerged):
    newWord = re.sub(' '.join(pairToBeMerged), ''.join(pairToBeMerged), word)
    return newWord

In [121]:
print(mergeLetters(newWord2, pairToBeMerged))

l o w es t _


In [135]:
def bpeTokenize(word, bpeStats):
    if len(word) == 1:
        return word
    word = ' '.join(list(word)) + ' _'
    while True:
        pairs = getAllPairs(word)
        pairToBeMerged = getPairToBeMerged(bpeStats, pairs)
        if pairToBeMerged == -1:
            break
        word = mergeLetters(word, pairToBeMerged)
    return word

In [137]:
pairToBeMerged

-1

In [138]:
newWord = bpeTokenize('lowest', bpeStats)

### Chat GPT - example

In [142]:
from tokenizers import ByteLevelBPETokenizer

# Carregar o tokenizer pré-treinado
tokenizer = ByteLevelBPETokenizer()
tokenizer.add_special_tokens(["<s>", "<pad>", "</s>"])

# Caminhos para os arquivos de treinamento
paths = ["./data/tokenizer_train/texto1.txt"]

# Treinamento do tokenizer BPE
tokenizer.train(files=paths, vocab_size=1000, min_frequency=2)

# Exemplo de codificação de texto
text = """
O Brasil é um país localizado na América do Sul. Possui uma extensa costa banhada pelo Oceano Atlântico. Sua capital é Brasília, mas a maior cidade do país é São Paulo. O Brasil é conhecido por sua diversidade cultural, belas praias, florestas tropicais e festas populares, como o Carnaval. Além disso, o país possui uma rica biodiversidade, abrigando a Floresta Amazônica, considerada um tesouro natural. A culinária brasileira também é bastante variada, com pratos típicos como feijoada, acarajé e brigadeiro. Venha conhecer as maravilhas do Brasil!
"""

encoded = tokenizer.encode(text)
decoded = tokenizer.decode(encoded.ids)

print("Texto original:", text)
print("Texto codificado:", encoded.tokens)
print("Texto decodificado:", decoded)


Texto original: 
O Brasil é um país localizado na América do Sul. Possui uma extensa costa banhada pelo Oceano Atlântico. Sua capital é Brasília, mas a maior cidade do país é São Paulo. O Brasil é conhecido por sua diversidade cultural, belas praias, florestas tropicais e festas populares, como o Carnaval. Além disso, o país possui uma rica biodiversidade, abrigando a Floresta Amazônica, considerada um tesouro natural. A culinária brasileira também é bastante variada, com pratos típicos como feijoada, acarajé e brigadeiro. Venha conhecer as maravilhas do Brasil!

Texto codificado: ['Ċ', 'O', 'ĠBrasil', 'ĠÃ©', 'Ġum', 'ĠpaÃŃs', 'Ġ', 'lo', 'c', 'al', 'iz', 'a', 'do', 'Ġna', 'ĠA', 'm', 'Ã©', 'r', 'ica', 'Ġdo', 'Ġ', 'S', 'ul', '.', 'ĠP', 'os', 'su', 'i', 'Ġuma', 'Ġe', 'xt', 'ensa', 'Ġco', 's', 'ta', 'Ġb', 'an', 'h', 'ada', 'Ġpel', 'o', 'ĠO', 'c', 'e', 'an', 'o', 'ĠA', 't', 'l', 'Ã¢', 'nt', 'ico', '.', 'Ġ', 'S', 'ua', 'Ġc', 'a', 'p', 'ita', 'l', 'ĠÃ©', 'ĠB', 'ras', 'ÃŃ', 'lia', ',', 'Ġma', '